In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv("/home/adnan/Documents/codes/data_Science/eda-and-visualization/IT-salary/data/processed/initial_cleaned_IT_survey.csv")
df.sample(3)

,timestamp,age,gender,city,position,experience,seniority_level,salary,salary_one_year_ago,language,company_size,company_type,survey_year,programming_language,vacation_days,сontract_duration
1530,11.12.2019 10:25:33,23.0,Female,Moscow,Data Engineer,4,Middle,32000.0,NaN,Russian,1000+,Product,2019,Hadoop,28.0,unlimited
381,16/12/2018 10:39:55,30.0,M,Berlin,Lead developer,9.0,Senior,75000.0,65000.0,English,50-100,Startup,2018,NaN,NaN,NaN
1375,06.12.2019 10:23:25,NaN,Female,Berlin,Manager,7,Senior,90000.0,NaN,English,1000+,Product,2019,Not Relevant,28.0,NaN


## Replacing Invalid DataPoints

- Unique Values from columns

In [3]:
na_columns = df.columns[(df.isna().mean() * 100 < 60) & df.isna().mean() * 100 > 0]

In [4]:
for each in na_columns:
    print(each)
    print(df[each].unique())
    print("-" * 10)

age
[43. 33. 32. 25. 39. 37. nan 36. 30. 40. 31. 29. 24. 44. 34. 28. 35. 38.
 27. 41. 26. 22. 42. 52. 50. 49. 23. 21. 48. 45. 60. 47. 46. 51. 54. 20.
 59. 66. 56. 53. 65. 69.]
----------
gender
['M' 'F' nan 'Male' 'Female' 'Diverse']
----------
city
['München' 'Berlin' 'Karlsruhe' 'Hamburg' 'Augsburg ' 'Bremen' 'Frankfurt'
 nan 'Regensburg' 'Lisbon' 'Leipzig' 'Stuttgart' 'Amsterdam' 'Luxembourg'
 'Bern' 'Bonn' 'Köln' 'Baden-Württemberg ' 'Frankonia' 'Stuttgart '
 'Mannheim ' 'Bayern' 'Tübingen' 'Brno' 'Dresden' 'Krakow' 'Dublin'
 'Eindhoven' 'Amsterdam ' 'Konstanz' 'Dublin ' 'Prague' 'Moscow'
 'Stockholm' 'Belgrade, Serbia' 'Kyiv' 'Nürnberg' 'Karlskrona' 'Krakau'
 'Vienna' 'London' 'Bitburg' 'Gdansk' 'Poland' 'Düsseldorf' 'Ludwigsburg'
 'Essen' 'Strasbourg' 'Alicante, Spain' 'Warsaw' 'Memmingen' 'Tallinn'
 'Zurich' 'oslo' 'Münster' 'Hildesheim' 'Krakow, Poland' 'Rosenheim'
 'Darmstadt' 'bremen' 'Brussels' 'Nürnberg ' 'Schleswig-Holstein' 'Riga'
 'Koblenz' 'homeoffice' 'Limassol' 'saint

In [5]:
df['experience'] = df['experience'].replace({"1 (as QA Engineer) / 11 in total" : '11', "15, thereof 8 as CTO" : '15',
                                            "6 (not as a data scientist, but as a lab scientist)" : '6', "383" : np.nan,
                                            "less than year" : '1', "2,5" : "2.5", "1,5" : "1.5"})

In [6]:
df['city'] = df['city'].replace({"Prefer not to say" : np.nan})

In [7]:
df['сontract_duration'] = df['сontract_duration'].replace({"Unlimited contract" : 'unlimited', "Temporary contract" : "temporary", 
                                 "0" : np.nan, "1 year" : '1Y', "more than 1 year" : '1Y+', '6 months' : '6M',
                                "3 months" : '3M', "less than 3 months" : '-3M'})

In [8]:
df['vacation_days'] = df['vacation_days'].replace({"30 in contract (but theoretically unlimited)" : "unlimited", '(no idea)' : np.nan,
                            "unlimited " : 'unlimited', 'Unlimited ' : 'unlimited', 'Unlimited' : 'unlimited',
                            '24 labour days' : '24', '23+' : '23', "~25" : '25'})

In [9]:
df['company_type'] = df['company_type'].replace({'eCommerce' : "E-commerce", "E-Commerce" : "E-commerce", 
                    "Ecommerce" : 'E-commerce', "e-commerce" : "E-commerce", "IT Consulting " : "IT Consultancy",
                    "IT Consultancy " : "IT Consultancy", "IT Consultants" : "IT Consultancy", "Consult" : "IT Consultancy",
                    "IT Consulting" : "IT Consultancy", "Consultancy" : "IT Consultancy", "Consulting" : "IT Consultancy",
                    "Full-time position in Education, part-time position in at a data startup" : "Education", 
                    "Consulting Company" : "IT Consultancy", "consulting" : "IT Consultancy", })

In [10]:
df['company_size'] = df['company_size'].replace({"101-1000" : "100-1000", "51-100" : "50-100", "11-50" : "10-50", 
                            "up to 10" : "upto-10"})

In [11]:
df['language'] = df['language'].replace({"Team - Russian; Cross-team - English;" : "Russian/English", 
                        'Russian, English' : "Russian/English", "Deutsch/Englisch" : "Deutsch/English",
                        'English+Deutsch' : "Deutsch/English", "Polish+English" : "Polish/English",
                        'English and German' : 'English/German', "50/50" : np.nan, "both" : np.nan,
                        "Deuglisch" : 'Deutsch/English'})

In [12]:
df['seniority_level'] = df['seniority_level'].replace({"no idea, there are no ranges in the firm " : np.nan, 
                        "student" : 'Student', 'intern' : 'Intern', "No level " : np.nan, "No level" : np.nan, 
                        "C-level executive manager" : 'C-Level', "Working Student" : "Student"})

In [13]:
df['gender'] = df['gender'].replace({"Male" : 'M', "Female" : 'F', "Diverse" : np.nan})

## Type convertion

In [14]:
def date_format(date: str) -> str:
    real = date.split(" ")
    result = real[0]
    result = "/".join(result.split("."))
    return result+ " " +real[1]

In [15]:
df['timestamp'] = df['timestamp'].apply(date_format)
df['age'] = df['age'].astype("Int8")
df['experience'] = df['experience'].astype("Float32").astype("Int8")
df['timestamp'] = pd.to_datetime(df['timestamp'], dayfirst=True)
df['survey_year'] = df['survey_year'].astype("int8")
df['salary'] = df['salary'].astype("float32")
df['salary_one_year_ago'] = df['salary_one_year_ago'].astype("float32")

## Handle Missing Values

In [16]:
mean_imputer = SimpleImputer()
mode_imputer = SimpleImputer(strategy="most_frequent")

In [17]:
(df.isna().mean() * 100).sort_values(ascending=False)

salary_one_year_ago     30.277685
programming_language    29.642021
vacation_days           29.240549
сontract_duration       26.898628
age                      7.059217
company_type             2.408832
seniority_level          1.137504
experience               1.070592
company_size             0.903312
city                     0.635664
language                 0.602208
position                 0.568752
gender                   0.368016
salary                   0.100368
timestamp                0.000000
survey_year              0.000000
dtype: float64

- Mean Imputer : for numerical variables

In [29]:
df['age'] = pd.Series(mean_imputer.fit_transform(df['age'].to_numpy().reshape(-1,1)).flatten()).astype(np.int8)
df['experience'] = pd.Series(mean_imputer.fit_transform(df['experience'].to_numpy().reshape(-1,1)).flatten()).astype(np.int8)

- Mode Imputer : for catagorical variables

In [53]:
df['gender'] = pd.Series(mode_imputer.fit_transform(df['gender'].to_numpy().reshape(-1, 1)).flatten())
df['company_type'] = pd.Series(mode_imputer.fit_transform(df['company_type'].to_numpy().reshape(-1, 1)).flatten())

- Impute With unknown/0

In [56]:
df = df.fillna({"city" : "unknown", "position" : "unknown", "seniority_level" : "unknown", "language" : "unknown",
          "company_size" : "unknown", "salary" : 0, "salary_one_year_ago" : 0})